In [43]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader



In [44]:
df = pd.read_csv('DATA/samsung_cospi.csv', index_col ='Date')

In [45]:
# data : 날짜
# Hign : 시가
# Low : 시가
# open : 시가
# adj close : 주식의 분할, 배당, 배분 등을 고려해 조정한 종가 -> 조정 종가 , Target values
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-04,6110.0,5660.0,6000.0,6110.0,74195000.0,4719.226074
2000-01-05,6060.0,5520.0,5800.0,5580.0,74680000.0,4309.866211
2000-01-06,5780.0,5580.0,5750.0,5620.0,54390000.0,4340.761230
2000-01-07,5670.0,5360.0,5560.0,5540.0,40305000.0,4278.971680
2000-01-10,5770.0,5580.0,5600.0,5770.0,46880000.0,4456.618652
...,...,...,...,...,...,...
2021-07-26,79500.0,78800.0,79400.0,78800.0,10040975.0,78800.000000
2021-07-27,79400.0,78500.0,79200.0,78500.0,11427693.0,78500.000000
2021-07-28,79200.0,78100.0,78300.0,79200.0,12743417.0,79200.000000


In [46]:
df.head(5) 

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-04,6110.0,5660.0,6000.0,6110.0,74195000.0,4719.226074
2000-01-05,6060.0,5520.0,5800.0,5580.0,74680000.0,4309.866211
2000-01-06,5780.0,5580.0,5750.0,5620.0,54390000.0,4340.761230
2000-01-07,5670.0,5360.0,5560.0,5540.0,40305000.0,4278.971680
2000-01-10,5770.0,5580.0,5600.0,5770.0,46880000.0,4456.618652


In [47]:
df['Adj Close'].plot(grid=True)

<AxesSubplot:title={'center':'Time-Series Prediction'}, xlabel='Date'>

In [48]:
X = df.drop(columns='Adj Close')
y = df.iloc[:, 5:6]
print(X)

               High      Low     Open    Close      Volume
Date                                                      
2000-01-04   6110.0   5660.0   6000.0   6110.0  74195000.0
2000-01-05   6060.0   5520.0   5800.0   5580.0  74680000.0
2000-01-06   5780.0   5580.0   5750.0   5620.0  54390000.0
2000-01-07   5670.0   5360.0   5560.0   5540.0  40305000.0
2000-01-10   5770.0   5580.0   5600.0   5770.0  46880000.0
...             ...      ...      ...      ...         ...
2021-07-26  79500.0  78800.0  79400.0  78800.0  10040975.0
2021-07-27  79400.0  78500.0  79200.0  78500.0  11427693.0
2021-07-28  79200.0  78100.0  78300.0  79200.0  12743417.0
2021-07-29  79400.0  78800.0  78800.0  79000.0  14360453.0
2021-07-30  78900.0  78500.0  78900.0  78500.0  13823411.0

[5419 rows x 5 columns]


In [49]:
print(y)

               Adj Close
Date                    
2000-01-04   4719.226074
2000-01-05   4309.866211
2000-01-06   4340.761230
2000-01-07   4278.971680
2000-01-10   4456.618652
...                  ...
2021-07-26  78800.000000
2021-07-27  78500.000000
2021-07-28  79200.000000
2021-07-29  79000.000000
2021-07-30  78500.000000

[5419 rows x 1 columns]


In [50]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()
ss = MinMaxScaler()


In [51]:
y_mm = mm.fit_transform(y) 
y_mm

array([[0.02963617],
       [0.02498908],
       [0.0253398 ],
       ...,
       [0.8751488 ],
       [0.87287838],
       [0.86720234]])

In [52]:
X_ss = ss.fit_transform(X)
X_ss

array([[0.03562314, 0.03720717, 0.03942571, 0.03829161, 0.45181622],
       [0.03509145, 0.03559945, 0.03714676, 0.0322873 , 0.45476966],
       [0.03211399, 0.03628847, 0.03657703, 0.03274046, 0.33121213],
       ...,
       [0.8128456 , 0.8690859 , 0.86326345, 0.86631925, 0.07760203],
       [0.81497235, 0.87712448, 0.8689608 , 0.86405347, 0.08744909],
       [0.80965547, 0.87367938, 0.87010027, 0.85838903, 0.08417874]])

In [53]:
X_train = X_ss[:4800, :]
X_test = X_ss[4800:, :]

In [54]:
y_train = y_mm[:4800, :]
y_test = y_mm[4800:, :]

In [55]:
print("Training Shape", X_train.shape, y_train.shape)
print("Testing Shape", X_test.shape, y_test.shape)


Training Shape (4800, 5) (4800, 1)
Testing Shape (619, 5) (619, 1)


In [56]:
X_train_tensors = torch.Tensor(X_train)
X_test_tensors = torch.Tensor(X_test)


In [57]:
y_train_tensors = torch.Tensor(y_train)
y_test_tensors = torch.Tensor(y_test)


In [58]:
X_train_tensors_final = torch.reshape(X_train_tensors, (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
X_test_tensors_final = torch.reshape(X_test_tensors, (X_test_tensors.shape[0], 1, X_test_tensors.shape[1]))


In [59]:
print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape)


Training Shape torch.Size([4800, 1, 5]) torch.Size([4800, 1])
Testing Shape torch.Size([619, 1, 5]) torch.Size([619, 1])


In [60]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
#device = torch.device("cpu")


In [61]:
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3080


In [62]:
num_epochs = 5000  ## 1000 epochs
learning_rate = 0.001

input_size = 5  # number of features
hidden_size = 2  # number of features in hidden state 
num_layers = 1  #number of stacked lstm layers               lstm layer 

num_classes = 1


In [63]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length    
        
        
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            batch_first = True)
        
        self.fc_1 = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128, num_classes)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x) #lstm with input, hidden, and internal state 
        # output.shape : [4800, 1, 2]
        # hn.shape : [1, 4800, 2]
        # cn.shape : [1, 4800, 2]
        
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next  , hn = (4800, 2)
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        
        return out
    


## Network 파라미터 

In [64]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1] ).to(device)

In [65]:
loss_function = torch.nn.MSELoss() # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) # adam optimizer



## Training

In [ ]:
for epoch in range(num_epochs):
    outputs = lstm1.forward(X_train_tensors_final.to(device)) #forward pass
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    
    #  the loss 
    loss = loss_function(outputs, y_train_tensors.to(device))
    loss.backward()
    
    optimizer.step() #improve from loss  i.e backpropagation

    if epoch % 100 == 0:
        print('Epoch : %d, loss : %.5f' %(epoch, loss.item()))

Epoch : 0, loss : 0.11540
Epoch : 100, loss : 0.01547
Epoch : 200, loss : 0.00400
Epoch : 300, loss : 0.00059
Epoch : 400, loss : 0.00036
Epoch : 500, loss : 0.00024
Epoch : 600, loss : 0.00018
Epoch : 700, loss : 0.00013
Epoch : 800, loss : 0.00009
Epoch : 900, loss : 0.00006
Epoch : 1000, loss : 0.00004
Epoch : 1100, loss : 0.00003
Epoch : 1200, loss : 0.00002
Epoch : 1300, loss : 0.00002
Epoch : 1400, loss : 0.00002
Epoch : 1500, loss : 0.00001
Epoch : 1600, loss : 0.00001
Epoch : 1700, loss : 0.00001
Epoch : 1800, loss : 0.00001
Epoch : 1900, loss : 0.00001
Epoch : 2000, loss : 0.00001
Epoch : 2100, loss : 0.00001
Epoch : 2200, loss : 0.00001
Epoch : 2300, loss : 0.00001


## Test & Analysis

In [ ]:
test_predict = lstm1(X_test_tensors_final.to(device))#forward pass

In [31]:
test_data_predict = test_predict.data.detach().cpu().numpy() #numpy conversion


In [32]:
test_data_predict

array([[0.43575972],
       [0.44174778],
       [0.4466645 ],
       [0.4483226 ],
       [0.4484551 ],
       [0.43644547],
       [0.4316371 ],
       [0.4404639 ],
       [0.4464323 ],
       [0.45373088],
       [0.4479106 ],
       [0.44810808],
       [0.44314456],
       [0.45107126],
       [0.45166224],
       [0.4520983 ],
       [0.45620447],
       [0.45420897],
       [0.45249993],
       [0.44046295],
       [0.43946052],
       [0.43024087],
       [0.42453668],
       [0.42777893],
       [0.42783588],
       [0.42511764],
       [0.4298547 ],
       [0.42581415],
       [0.42229256],
       [0.42327657],
       [0.42331773],
       [0.4221339 ],
       [0.42412344],
       [0.43861896],
       [0.4498574 ],
       [0.4381262 ],
       [0.43774456],
       [0.43630314],
       [0.4337507 ],
       [0.42996693],
       [0.43581742],
       [0.44120646],
       [0.44870168],
       [0.4498574 ],
       [0.4538139 ],
       [0.45299345],
       [0.45065522],
       [0.448

In [33]:
test_data_predict = mm.inverse_transform(test_data_predict) #reverse transformation
y_test = mm.inverse_transform(y_test)


In [34]:
plt.figure(figsize=(10,6)) #plotting 
plt.plot(y_test, label='Actuall Data') # True plot
plt.plot(test_data_predict, label='Predicted Data') # predicted plot
plt.title('Time-Series Prediction')


Text(0.5, 1.0, 'Time-Series Prediction')

In [35]:
# 3000 

In [36]:
df_X_ss = ss.transform(df.drop(columns='Adj Close')) 
df_y_mm = mm.transform(df.iloc[:, 5:6])
df_X_ss = torch.Tensor(df_X_ss) #converting to Tensors
df_y_mm = torch.Tensor(df_y_mm) #converting to Tensors

df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], 1, df_X_ss.shape[1]))



In [37]:
df_X_ss

tensor([[[0.0356, 0.0372, 0.0394, 0.0383, 0.4518]],

        [[0.0351, 0.0356, 0.0371, 0.0323, 0.4548]],

        [[0.0321, 0.0363, 0.0366, 0.0327, 0.3312]],

        ...,

        [[0.8128, 0.8691, 0.8633, 0.8663, 0.0776]],

        [[0.8150, 0.8771, 0.8690, 0.8641, 0.0874]],

        [[0.8097, 0.8737, 0.8701, 0.8584, 0.0842]]])

In [38]:
#df_y_mm.numpy()

In [39]:
train_predict = lstm1(df_X_ss.to(device))#forward pass

data_predict = train_predict.data.detach().cpu().numpy() #numpy conversion
dataY_plot = df_y_mm.data.numpy() #



In [40]:
data_predict = mm.inverse_transform(data_predict) #reverse transformation 
dataY_plot = mm.inverse_transform(dataY_plot)


In [41]:
data_predict

array([[ 4627.03  ],
       [ 4451.0283],
       [ 4390.741 ],
       ...,
       [64599.18  ],
       [64590.754 ],
       [64373.562 ]], dtype=float32)

In [42]:


plt.figure(figsize=(10,6)) #plotting 
plt.axvline(x=4800, c='r', linestyle='--') 
plt.plot(dataY_plot, label='Actuall Data') # True plot
plt.plot(data_predict, label='Predicted Data') # predicted plot
plt.title('Time-Series Prediction')



Text(0.5, 1.0, 'Time-Series Prediction')